In [1]:
import json 
import copy
import pickle
import os
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from matplotlib.path import Path
from torchvision import transforms
from torch.utils.data import Dataset
from skimage import draw
from skimage import io
import re

import torch
import numpy as np
import pandas as pd
plt.rcParams["figure.figsize"] = (15,8)
from skimage.io import imread, imshow, imsave
from skimage.color import rgb2gray
from skimage.draw import polygon
from skimage import draw
from sklearn.metrics import confusion_matrix
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon


mappings = {
	'Thick reticular or branched lines': 'TRBL',
	'Dicke retikuläre oder verzweigte Linien': 'TRBL',

	'Eccentrically located structureless area (any colour except skin colour, white and grey)': 'ESA',
	'Exzentrisch gelegenes, strukturloses Areal jeglicher Farbe, außer hautfarben, weiß und grau': 'ESA',

	'Grey patterns': 'GP',
	'Graue Muster': 'GP',

	'Polymorphous vessels': 'PV',
	'Polymorphe Gefäße': 'PV',

	'Pseudopods or radial lines at the lesion margin that do not occupy the entire lesional circumference': 'PRL',
	'Pseudopodien oder radiale Linien am Läsionsrand, die nicht den gesamten Läsionsumfang einnehmen': 'PRL',

	'Black dots or globules in the periphery of the lesion': 'BDG',
	'Schwarze Punkte oder Schollen in der Läsionsperipherie': 'BDG',

	'White lines or white structureless area': 'WLSA',
	'Weiße Linien oder weißes strukturloses Areal': 'WLSA',

	'Parallel lines on ridges (acral lesions only)': 'PLR',
	'Parallele Linien auf den Leisten (nur akrale Läsionen)': 'PLR',

	'Pigmentation extends beyond the area of the scar (only after excision)': 'PES',
	'Pigmentierung überschreitet Narbenareal (nur nach Entfernung)': 'PES',

	'Pigmentation invades the openings of hair follicles (facial lesions)': 'PIF',
	'Pigmentierung überschreitet Follikelöffnung (Gesichtsläsionen)': 'PIF',

	'Only one pattern and only one colour': 'OPC',
	'Nur ein Muster und nur eine Farbe': 'OPC',

	'Symmetrical combination of patterns and;or colours': 'SPC',
	'Symmetrische Kombination von Mustern und;oder Farben': 'SPC',
    '\t\tSymmetrical combination of patterns and;or colours': 'SPC',

	'Monomorphic vascular pattern': 'MVP',
	'Monomorphes Gefäßmuster': 'MVP',

	'Pseudopods or radial lines at the lesional margin involving the entire lesional circumference': 'PRLC',
	'Pseudopodien oder radiale Linien am Läsionsrand über den gesamten Läsionsumfang': 'PRLC',

	'Parallel lines in the furrows (acral lesions only)': 'PLF',
	'Parallele Linien in den Furchen (nur akrale Läsionen)': 'PLF',

	'Pigmentation does not extend beyond the area of the scar (only after excision)': 'PDES',
	'Pigmentierung überschreitet Narbe nicht (nur nach Entfernung)': 'PDES',

	'Asymmetric combination of multiple patterns and;or colours in the absence of other melanoma criteria': 'APC',
	'Asymmetrische Kombination mehrerer Muster und;oder Farben ohne weitere Melanomkriterien': 'APC',

	'Melanoma simulator': 'MS',
	'Melanomsimulator': 'MS',
    
	'Please select at least one explanation.': 'XX',
	'Bitte wählen Sie mindestens eine Erklärung aus.': 'XX',
    'Andere (bitte angeben)': 'XX'
}

char_class_labels = ['TRBL', 'ESA', 'BDG', 'GP', 'PV', 'PRL', 'WLSA', 'PES', 'PIF', 'OPC', 'SPC', 'MVP', 'PRLC', 'PLF', 'PDES', 'APC', 'MS']
mel_class_labels = ['TRBL', 'ESA', 'BDG', 'GP', 'PV', 'PRL', 'WLSA', 'PES', 'PIF', 'PLR']
nv_class_labels = ['OPC', 'SPC', 'MVP', 'PRLC', 'PLF', 'PDES', 'APC', 'MS']

def process_annotation(polygons):
    # Sometimes the polygon label was defined as bounding box (human error),
    #  the 'points' key is missing in that case, so the following if condition handles that.
    if 'points' in polygons['data']:
        polygon = np.array(polygons['data']['points'])
    else:
        botton_left = polygons['data']['min']
        top_right = polygons['data']['max']
        botton_right = [top_right[0], botton_left[1]]
        top_left = [botton_left[0], top_right[1]]
        polygon = np.array([botton_left, botton_right, top_right, top_left])

    polygon[:, 0] *= 600
    polygon[:, 1] *= 450
    polygon = polygon.astype(int)
    
    if 'Melanoma' in polygons['children']:
        polygons['children']['Explanation'] = polygons['children'].pop('Melanoma')
    
    # Different spellings of Explanation, sometimes even the English word was defined
    try:
        explanation = polygons['children']['Erklärung']['data']['selected']
    except:
        try:
            explanation = polygons['children']['Explanation']['data']['selected']
        except: 
            explanation = polygons['children']['Explanation']['data']['selected']
                
    try:
        confidence = polygons['children']['Confidence']['data']['selected']
    except:
        confidence = -1
    if confidence == 'Please select a confidence.':
        confidence = -1
    else:
        confidence = int(confidence)
    
    if type(explanation) != list:
        explanation = [explanation]
        
    explanation = [re.sub('\t+', '', exp) for exp in explanation]
   
    explanation = [mappings[exp] for exp in explanation]
    
    # XX has been mapped from "Please choose 1 explanation" and "Other Explanation"
    # Since XX is auto selected in every explanation we're dropping XX
    # Unless XX is the only explanation, which means no explanation was chosen,
    #  renaming it to 'None'
    while 'XX' in explanation: explanation.remove('XX')
    
    if len(explanation) == 0:
        explanation.append('None')
    
    return polygon, explanation, confidence


/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Process metadata files individually in a loop

In [2]:
#ham = pd.read_csv("/home/kti01/Documents/My Files/Data/HAM10000/HAM10000/metadata")
#ham = ham[ham.dx.isin(['mel', 'nv'])]
#ham = ham[ham.dx_type=='histo']

ham = pd.read_csv("/home/kti01/Documents/My Files/Projects/Overlap/data/metadata_testset.csv")

# CHANGE THIS DIR
phase1_dir = "/home/kti01/Documents/My Files/Projects/Overlap/data/phase1/english"
participants = os.listdir(phase1_dir)
participants = [x for x in participants if not x.endswith('.pkl')]
confidences = {}

for participant in participants:
    data_dir = os.path.join(phase1_dir, participant)
    files = []
    for file in ['train.json', 'validation.json', 'test.json']:

        data_file = os.path.join(data_dir, file)

        with open(os.path.join(data_file)) as f:
            d = json.load(f)
            annotations = dict(d)['labels']

        # Store labels in a dict with image names as keys and labels as values
        labels = {annotation['dataId']: annotation['annotations'] for annotation in annotations}


        image_list = []
        polygons_list = []
        explanations_list = []
        confidence_list = []
        for k, v in labels.items():   
            if len(v) == 0:
                continue
            
            if len(v['Melanoma']) != 0:
                for polygons in v['Melanoma']:

                    polygon, explanation, confidence = process_annotation(polygons)

                    image_list.append(k)
                    polygons_list.append(polygon)
                    explanations_list.append(explanation)
                    confidence_list.append(confidence)
            elif len(v['Nevus (leave in)']) != 0:
                for polygons in v['Nevus (leave in)']:
                    polygon, explanation, confidence = process_annotation(polygons)

                    image_list.append(k)
                    polygons_list.append(polygon)
                    explanations_list.append(explanation)
                    confidence_list.append(confidence)
            else:
                for polygons in v['Nevus (excise)']:
                    polygon, explanation, confidence = process_annotation(polygons)

                    image_list.append(k)
                    polygons_list.append(polygon)
                    explanations_list.append(explanation)
                    confidence_list.append(confidence)
        """
        confidences[participant] = {file: ''}
        confidences[participant][file] = pd.DataFrame({'image_id': image_list, 'explanation': explanations_list, 
                                                       'confidence': confidence_list})
        """

        labels = pd.DataFrame({'image_id': image_list, 'polygon': polygons_list, 'explanation': explanations_list,
                              'confidence': confidence_list})
        #labels['explanation'] = labels['explanation'].apply(lambda x: ['Bitte wählen Sie mindestens eine Erklärung aus.'] if len(x)==0 else x)
        
        metadata_df = pd.DataFrame(labels.explanation.explode())
        metadata_df = pd.merge(labels['image_id'], metadata_df, left_index=True, right_index=True)
        metadata_df = pd.get_dummies(metadata_df.explanation)


        metadata_df = labels.join(metadata_df)
        metadata_df = metadata_df.groupby('image_id').mean()
        metadata_df = metadata_df.reset_index()
            
            
        for col in metadata_df.columns.drop(['image_id', 'confidence']):
            metadata_df.loc[metadata_df[col]>0, col] = 1 

        metadata_df['mask'] = metadata_df['image_id'].apply(lambda x: x.split('.')[0]).astype(np.int64)
        metadata_df.drop('image_id', axis=1, inplace=True)

        metadata_df = pd.merge(ham, metadata_df, on='mask', how='inner')
        metadata_df['benign_malignant'] = metadata_df['dx'].apply(lambda x: 0 if x == 'nv' else 1)
        one_hot = pd.get_dummies(metadata_df['dx'])
        metadata_df = metadata_df.join(one_hot)



        labels['mask'] = labels['image_id'].apply(lambda x: x.split('.')[0]).astype(np.int64)
        labels.drop('image_id', axis=1, inplace=True)
        labels = pd.merge(labels, metadata_df[['mask', 'image_id']], on='mask', how='inner')
        labels.drop('mask', axis=1, inplace=True)


        # Add polygons as a column
        annotations_dict = labels.set_index('image_id', drop=True).groupby(level=0).apply(lambda x: x.to_dict('records')).to_dict()
        metadata_df['annotations'] = [annotations_dict[img] for img in metadata_df.image_id]
        metadata_df['image'] = metadata_df['image_id'].apply(lambda x: x+'.jpg')


        # Put each explanation and it's corresponding polygon in individual rows
        d = {'image_id': [], 'explanation': [], 'polygon': []}
        for idx, row in labels.iterrows():
            for exp in row['explanation']:
                d['image_id'].append(row['image_id'])
                d['explanation'].append(exp)
                d['polygon'].append(row['polygon'])
        d = pd.DataFrame(d)

        final_dict = {'image_id': [], 'explanation': [], 'polygon': []}
        # Loop over each image
        for image_id in d['image_id'].unique():
            # Get sub dataframe consisting of only one image's explanations 
            sub_d = d[d['image_id'] == image_id]
            # Loop over each explanation for the current image
            for exp in sub_d['explanation'].unique():
                # Select all rows of the dataframe where the explanation is the current explanation
                sub_de = sub_d[sub_d['explanation'] == exp]
                # Loop over each polygon and append to list
                poly_list = []
                for poly in sub_de['polygon']:
                    poly_list.append(poly)

                # Keep appending to lists in a dict so that we can create a dataframe out of this
                final_dict['polygon'].append(poly_list)
                final_dict['image_id'].append(image_id)
                final_dict['explanation'].append(exp)
                
        final_df = pd.DataFrame(final_dict)
        final_df = final_df.pivot_table(values='polygon', index=final_df['image_id'], columns='explanation', aggfunc='first')

        final_df.fillna(-1, inplace=True)

        # Rename columns
        for col in final_df.columns:
            final_df.rename(columns={col: col+'_annotation'}, inplace=True)
        final_df['participant'] = participant

        final_merged_df = pd.merge(metadata_df, final_df, on='image_id', how='inner')
        files.append(final_merged_df)
        
        
    final_final_merged_df = pd.concat(files)
    final_final_merged_df.reset_index(drop=True, inplace=True)
    final_final_merged_df.fillna(-1, inplace=True)
    
    # Infer prediction
    current_mel_labels = list(set(mel_class_labels).intersection(final_final_merged_df.columns))
    final_final_merged_df['prediction'] = final_final_merged_df.apply(lambda x: 1 if (x[current_mel_labels]==1).any() else 0, axis=1)
    
    # Reorder columns
    #final_final_merged_df = final_final_merged_df[ ['participant'] + [ col for col in final_final_merged_df.columns if col != 'participant' ] ]
    
    with open(os.path.join(data_dir, 'metadata.pkl'), 'wb') as fp:
        pickle.dump(final_final_merged_df, fp, protocol=pickle.HIGHEST_PROTOCOL)
    print('Saved metadata for '+participant)
    

/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning

Saved metadata for e79
Saved metadata for e10
Saved metadata for e66


/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning

Saved metadata for e77
Saved metadata for e19
Saved metadata for e76


/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning

...

/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning

Saved metadata for e48
Saved metadata for e72
Saved metadata for e21
Saved metadata for e56


/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():
/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning

Saved metadata for e31
Saved metadata for e24
Saved metadata for e64
Saved metadata for e81


/home/kti01/miniconda3/envs/artifact_detection/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


# Merge all individual annotator's metadata files into one file

In [29]:

participants = os.listdir(phase1_dir)
participants = [x for x in participants if not x.endswith('.pkl') and not x.endswith('.csv')]

metadata_list = []
for participant in participants:
    data_dir = os.path.join(phase1_dir, participant)
    
    metadata = pd.read_pickle(os.path.join(data_dir, 'metadata.pkl'))
    metadata['participant'] = participant
    metadata_list.append(metadata)
metadata_df = pd.concat(metadata_list)
metadata_df.fillna(-1, inplace=True)
metadata_df = metadata_df[['participant'] + [col for col in metadata_df.columns if col != 'participant' ]]
metadata_df['confidence'] = metadata_df['confidence'].round()

In [30]:
with open(os.path.join(phase1_dir, 'metadata_phase1_english.pkl'), 'wb') as fp:
    pickle.dump(metadata_df, fp, protocol=pickle.HIGHEST_PROTOCOL)
    print('Saved consolidated metadata file')

Saved consolidated metadata file


## Concat german and english metadata into one file

In [33]:
de_metadata = pd.read_pickle("/home/kti01/Documents/My Files/Projects/Overlap/data/phase1/german/metadata_phase1_german.pkl")
en_metadata = pd.read_pickle("/home/kti01/Documents/My Files/Projects/Overlap/data/phase1/english/metadata_phase1_english.pkl")
de_metadata['language'] = 'de'
en_metadata['language'] = 'en'
metadata_phase1 = pd.concat([de_metadata, en_metadata])

AI_predictions = pd.read_csv("/home/kti01/Documents/My Files/Projects/Overlap/data/AI_predictions.csv")
metadata_phase1 = pd.merge(metadata_phase1, AI_predictions[['group', 'mask', 'AI_prediction']], how='left', left_on=['group', 'mask'], right_on=['group', 'mask'])

# Save pickle
with open("/home/kti01/Documents/My Files/Projects/Overlap/data/phase1/metadata_phase1.pkl", 'wb') as fp:
    pickle.dump(metadata_phase1, fp, protocol=pickle.HIGHEST_PROTOCOL)
    print('Saved consolidated metadata file')

# Save csv
metadata_phase1 = metadata_phase1[['participant', 'group', 'image_id', 'mask', 'benign_malignant', 'prediction', 'confidence', 'AI_prediction', 'language']]
metadata_phase1['group'] = metadata_phase1['group'].astype(int)
metadata_phase1['mask'] = metadata_phase1['mask'].astype(int)
metadata_phase1['confidence'] = metadata_phase1['confidence'].astype(int)
metadata_phase1.to_csv("/home/kti01/Documents/My Files/Projects/Overlap/data/phase1/metadata_phase1.csv", index=False)
print('Saved consolidated metadata csv')

Saved consolidated metadata file
Saved consolidated metadata csv
